In [2]:
#Import
import os
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA # this will make question and answering
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone


# I am going to import my PDF so i need to import DirectoryLoader
from langchain.document_loaders import PyPDFLoader,DirectoryLoader

# Corpus to Chunks
from langchain.text_splitter import RecursiveCharacterTextSplitter


# we are using quant size model so we have to import ctransformer
from langchain.llms import CTransformers

c:\Users\arung\anaconda3\envs\chatbot\lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [9]:
key=os.getenv("PINECONE_API_KEY")

In [11]:
# Dont use mine because i will delete it after completion of my project
PINECONE_API_KEY="1a0b20f7-7500-40fe-823a-dbdb343f593f"
PINECONE_API_ENV="gcp-starter"

# Load PDF

In [12]:
#load my pdf using Directory Loader
def load_pdf(data):
    loader=DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    documents=loader.load()
    return documents
